In [1]:
%%time
!pip uninstall -y torch
!pip install -q --no-index --find-links=/kaggle/input/making-wheels-of-necessary-packages-for-vllm vllm
!pip install -q -U /kaggle/input/vllm-t4-fix/grpcio-1.62.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!pip install -q -U /kaggle/input/vllm-t4-fix/ray-2.11.0-cp310-cp310-manylinux2014_x86_64.whl
!pip install -q --no-deps --no-index /kaggle/input/hf-libraries/sentence-transformers/sentence_transformers-3.1.0-py3-none-any.whl
!pip install --no-deps --no-index /kaggle/input/logits-processor-zoo/logits_processor_zoo-0.1.0-py3-none-any.whl
!pip install transformers peft accelerate -q -U --no-index --find-links /kaggle/input/lmsys-wheel-files
!pip install --no-index /kaggle/input/bitsandbytes0-42-0/bitsandbytes-0.42.0-py3-none-any.whl --find-links=/kaggle/input/bitsandbytes0-42-0
!pip install --no-index  /kaggle/input/bitsandbytes0-42-0/optimum-1.21.2-py3-none-any.whl --find-links=/kaggle/input/bitsandbytes0-42-0
!pip install --no-index  /kaggle/input/bitsandbytes0-42-0/auto_gptq-0.7.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl --find-links=/kaggle/input/bitsandbytes0-42-0


Found existing installation: torch 2.4.0
Uninstalling torch-2.4.0:
  Successfully uninstalled torch-2.4.0
Processing /kaggle/input/logits-processor-zoo/logits_processor_zoo-0.1.0-py3-none-any.whl
Looking in links: /kaggle/input/bitsandbytes0-42-0
Processing /kaggle/input/bitsandbytes0-42-0/bitsandbytes-0.42.0-py3-none-any.whl
Looking in links: /kaggle/input/bitsandbytes0-42-0
Processing /kaggle/input/bitsandbytes0-42-0/optimum-1.21.2-py3-none-any.whl
Processing /kaggle/input/bitsandbytes0-42-0/coloredlogs-15.0.1-py2.py3-none-any.whl (from optimum==1.21.2)
Processing /kaggle/input/bitsandbytes0-42-0/transformers-4.42.4-py3-none-any.whl (from transformers[sentencepiece]<4.43.0,>=4.26.0->optimum==1.21.2)
Processing /kaggle/input/bitsandbytes0-42-0/tokenizers-0.19.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (from transformers<4.43.0,>=4.26.0->transformers[sentencepiece]<4.43.0,>=4.26.0->optimum==1.21.2)
Processing /kaggle/input/bitsandbytes0-42-0/humanfriendly-10.0-py2.py3

In [2]:
# Basic Python Libraries
import os
import sys
import warnings
import math

# Data Processing Libraries
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors

# PyTorch and Related
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import Tensor
from torch.utils.data import Dataset, DataLoader

# Transformers and PEFT
from transformers import (
    AutoTokenizer,
    AutoModel,
    AutoConfig,
    AutoModelForMaskedLM,
    BitsAndBytesConfig
)
from peft import (
    LoraConfig,
    get_peft_model
)

# Progress Bars
from tqdm import tqdm
from tqdm.autonotebook import trange

warnings.filterwarnings('ignore')

path_prefix = "/kaggle/input/eedi-mining-misconceptions-in-mathematics"
model_path = "/kaggle/input/sfr-embedding-mistral/SFR-Embedding-2_R"
lora_path="/kaggle/input/v7-recall/epoch_19_model/adapter.bin"
device='cuda:0'

In [3]:
full_df = pd.read_csv("/kaggle/input/eedi-mining-misconceptions-in-mathematics/test.csv")


rows = []
for idx, row in full_df.iterrows():
    for option in ["A", "B", "C", "D"]:
        if option == row.CorrectAnswer:
            continue
            
        correct_answer = row[f"Answer{row.CorrectAnswer}Text"]

        query_text =f"###question###:{row['SubjectName']}-{row['ConstructName']}-{row['QuestionText']}\n###Correct Answer###:{correct_answer}\n###Misconcepte Incorrect answer###:{option}.{row[f'Answer{option}Text']}"

        rows.append({"query_text": query_text, 
                     "QuestionId_Answer": f"{row.QuestionId}_{option}",
                     "ConstructName": row.ConstructName,
                     "SubjectName": row.SubjectName,
                     "QuestionText": row.QuestionText,
                     "correct_answer": correct_answer,
                     "incorrect_answer": row[f"Answer{option}Text"]
                     })

df = pd.DataFrame(rows)

In [4]:
def batch_to_device(batch, target_device):
    """
    send a pytorch batch to a device (CPU/GPU)
    """
    for key in batch:
        if isinstance(batch[key], Tensor):
            batch[key] = batch[key].to(target_device)
    return batch

def last_token_pool(last_hidden_states: Tensor,
                    attention_mask: Tensor) -> Tensor:
    left_padding = (attention_mask[:, -1].sum() == attention_mask.shape[0])
    if left_padding:
        return last_hidden_states[:, -1]
    else:
        sequence_lengths = attention_mask.sum(dim=1) - 1
        batch_size = last_hidden_states.shape[0]
        return last_hidden_states[torch.arange(batch_size, device=last_hidden_states.device), sequence_lengths]

def get_detailed_instruct(task_description: str, query: str) -> str:
    return f'Instruct: {task_description}\nQuery: {query}'

def inference(df, model, tokenizer, device):
    batch_size = 16
    max_length = 512
    sentences = list(df['query_text'].values)

    all_embeddings = []
    length_sorted_idx = np.argsort([-len(sen) for sen in sentences])
    sentences_sorted = [sentences[idx] for idx in length_sorted_idx]
    for start_index in trange(0, len(sentences), batch_size, desc="Batches", disable=False):
        sentences_batch = sentences_sorted[start_index: start_index + batch_size]
        features = tokenizer(sentences_batch, max_length=max_length, padding=True, truncation=True,
                             return_tensors="pt")
        features = batch_to_device(features, device)
        with torch.no_grad():
            outputs = model.model(**features)
            embeddings = last_token_pool(outputs.last_hidden_state, features['attention_mask'])
            embeddings = torch.nn.functional.normalize(embeddings, dim=-1)
            embeddings = embeddings.detach().cpu().numpy().tolist()
        all_embeddings.extend(embeddings)

    all_embeddings = [np.array(all_embeddings[idx]).reshape(1, -1) for idx in np.argsort(length_sorted_idx)]

    return np.concatenate(all_embeddings, axis=0)

In [5]:
tokenizer = AutoTokenizer.from_pretrained(lora_path.replace("/adapter.bin",""))
bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.bfloat16
        )
backbone = AutoModel.from_pretrained(model_path, quantization_config=bnb_config,device_map=device)
config = LoraConfig(
        r=64,
        lora_alpha=256,
        target_modules=[
            "q_proj",
            "k_proj",
            "v_proj",
            "o_proj",
            "gate_proj",
            "up_proj",
            "down_proj",
        ],
        bias="none",
        lora_dropout=0.05,  # Conventional
        task_type="CAUSAL_LM",
    )
model = get_peft_model(backbone, config)
d = torch.load(lora_path, map_location=model.device)
model.load_state_dict(d, strict=False)
model = model.eval()
model = model.to(device)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
task_description = 'Given a math question and a misconcepte incorrect answer, please retrieve the most accurate reason for the misconception. You need to thought carefully and reasonal'


V_answer = inference(df, model, tokenizer, device)

misconception_df = pd.read_csv("/kaggle/input/eedi-mining-misconceptions-in-mathematics/misconception_mapping.csv")
misconception_df["query_text"] = misconception_df["MisconceptionName"]

V_misconception = inference(misconception_df, model, tokenizer, device)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/162 [00:00<?, ?it/s]

In [7]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"

def get_matches(V_topic, V_content, n_neighbors=25):
    
    neighbors_model = NearestNeighbors(n_neighbors=n_neighbors, metric='cosine', algorithm="brute", n_jobs=-1)
    neighbors_model.fit(V_content)
    dists, indices = neighbors_model.kneighbors(V_topic)
    
    return indices

indices = get_matches(V_answer, V_misconception, n_neighbors=25)
indices.shape

(9, 25)

In [8]:
import gc

del backbone, model

gc.collect()
torch.cuda.empty_cache()

np.save("indices.npy", indices)
df.to_parquet("df.parquet", index=False)

In [9]:
%%writefile run_vllm.py

import vllm
import numpy as np
import pandas as pd
from transformers import PreTrainedTokenizer, AutoTokenizer
from typing import List
import torch
from logits_processor_zoo.vllm import MultipleChoiceLogitsProcessor
import re

model_path = "/kaggle/input/m/abdullahmeda/qwen2.5/transformers/math-7b-instruct/1"
tokenizer = AutoTokenizer.from_pretrained(model_path)


def preprocess_text(x):
    x = re.sub("http\w+", '',x)   # Delete URL
    x = re.sub(r"\.+", ".", x)    # Replace consecutive commas and periods with one comma and period character
    x = re.sub(r"\,+", ",", x)
    x = re.sub(r"\\\(", " ", x)
    x = re.sub(r"\\\)", " ", x)
    x = re.sub(r"[ ]{1,}", " ", x)
    x = x.strip()                 # Remove empty characters at the beginning and end
    return x

PROMPT  = """Here is a question about {ConstructName}({SubjectName}).
Question: {Question}
Correct Answer: {CorrectAnswer}
Incorrect Answer: {IncorrectAnswer}

You are a Mathematics teacher. Your task is to reason and identify the misconception behind the Incorrect Answer with the Question.
Answer concisely what misconception it is to lead to getting the incorrect answer.
Pick the correct misconception number from the below:

{Retrival}
"""
# just directly give your answers.

def apply_template(row, tokenizer):
    messages = [
        {
            "role": "user", 
            "content": preprocess_text(
                PROMPT.format(
                    ConstructName=row["ConstructName"],
                    SubjectName=row["SubjectName"],
                    Question=row["QuestionText"],
                    IncorrectAnswer=row[f"incorrect_answer"],
                    CorrectAnswer=row[f"correct_answer"],
                    Retrival=row[f"retrieval"]
                )
            )
        }
    ]
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return text


misconception_df = pd.read_csv("/kaggle/input/eedi-mining-misconceptions-in-mathematics/misconception_mapping.csv")

df = pd.read_parquet("df.parquet")
indices = np.load("indices.npy")

llm = vllm.LLM(
    model_path,
    # quantization="awq",
    tensor_parallel_size=2,
    gpu_memory_utilization=0.90, 
    trust_remote_code=True,
    dtype="half", 
    enforce_eager=True,
    max_model_len=4096,
    disable_log_stats=True
)
tokenizer = llm.get_tokenizer()


def get_candidates(c_indices):
    candidates = []

    mis_names = misconception_df["MisconceptionName"].values
    for ix in c_indices:
        c_names = []
        for i, name in enumerate(mis_names[ix]):
            c_names.append(f"{i+1}. {name}")

        candidates.append("\n".join(c_names))
        
    return candidates

survivors = indices[:, -1:]

for i in range(3):
    c_indices = np.concatenate([indices[:, -8*(i+1)-1:-8*i-1], survivors], axis=1)
    
    df["retrieval"] = get_candidates(c_indices)
    df["text"] = df.apply(lambda row: apply_template(row, tokenizer), axis=1)
    
    print("Example:")
    print(df["text"].values[0])
    print()
    
    responses = llm.generate(
        df["text"].values,
        vllm.SamplingParams(
            n=1,  # Number of output sequences to return for each prompt.
            top_k=1,  # Float that controls the cumulative probability of the top tokens to consider.
            temperature=0,  # randomness of the sampling
            seed=777, # Seed for reprodicibility
            skip_special_tokens=False,  # Whether to skip special tokens in the output.
            max_tokens=1,  # Maximum number of tokens to generate per output sequence.
            logits_processors=[MultipleChoiceLogitsProcessor(tokenizer, choices=["1", "2", "3", "4", "5", "6", "7", "8", "9"])]
        ),
        use_tqdm=True
    )
    
    responses = [x.outputs[0].text for x in responses]
    df["response"] = responses
    
    
    llm_choices = df["response"].astype(int).values - 1
    
    survivors = np.array([cix[best] for best, cix in zip(llm_choices, c_indices)]).reshape(-1, 1)



results = []

for i in range(indices.shape[0]):
    ix = indices[i]
    llm_choice = survivors[i, 0]
    
    results.append(" ".join([str(llm_choice)] + [str(x) for x in ix if x != llm_choice]))


df["MisconceptionId"] = results
df.to_csv("submission.csv", columns=["QuestionId_Answer", "MisconceptionId"], index=False)
print('finish write file')

Writing run_vllm.py


In [10]:
!python run_vllm.py

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
WARNING 12-05 14:55:32 config.py:1425] Casting torch.bfloat16 to torch.float16.
INFO 12-05 14:55:32 config.py:715] Defaulting to use mp for distributed inference
INFO 12-05 14:55:32 llm_engine.py:176] Initializing an LLM engine (v0.5.3.post1) with config: model='/kaggle/input/m/abdullahmeda/qwen2.5/transformers/math-7b-instruct/1', speculative_config=None, tokenizer='/kaggle/input/m/abdullahmeda/qwen2.5/transformers/math-7b-instruct/1', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=2, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=True, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_conf

In [11]:
pd.read_csv("submission.csv")

,QuestionId_Answer,MisconceptionId
0,1869_B,1672 2532 706 1941 1054 77 1507 2586 1516 1345...
1,1869_C,2306 2488 1941 655 1054 1689 117 375 1135 1372...
2,1869_D,1672 2532 158 77 1941 15 706 2586 1345 1392 11...
3,1870_A,2142 2068 143 2372 2078 418 320 2139 1904 1606...
4,1870_B,143 891 1871 363 1593 2549 413 1540 2078 979 1...
5,1870_C,2142 143 2068 418 2372 1872 1416 715 1319 1432...
6,1871_A,2408 1408 1923 1287 1073 397 2584 906 941 1498...
7,1871_C,2408 1408 1287 1923 1073 2584 2188 906 941 912...
8,1871_D,1073 1408 2408 906 1287 2439 2188 1923 2177 15...
